In [ ]:
import pathlib
import json

In [ ]:
from textrec.paths import paths

In [ ]:
import glob
import os
import json
import dateutil.parser
import datetime
import toolz

In [ ]:
def get_log_data(log_file, earliest):
    size = os.path.getsize(log_file)
    meta = None
    num_nexts = 0
    with open(log_file) as f:
        for idx, line in enumerate(f):
            if 'next' not in line and 'login' not in line and 'finalData' not in line:
                continue
            line = json.loads(line)
            if line.get('type') == 'next':
                num_nexts += 1
            elif line.get('type') == 'login':
                if 'jsTimestamp' in line:
                    timestamp = datetime.datetime.fromtimestamp(line['jsTimestamp'] / 1000)
                else:
                    timestamp = dateutil.parser.parse(line['timestamp'])
                if timestamp < earliest:
                    return
                platform_id = line['platform_id']
                meta = dict(timestamp=timestamp, config=line['config'], platform_id=platform_id, participant_id=line['participant_id'], size=size)
            elif line.get('type') == 'finalData':
#                 meta['finalData'] = line['finalData']
                meta['complete'] = True
    if meta:
        return dict(meta, num_nexts=num_nexts)


def get_logs(log_path, earliest):
    log_files = []
    for log_file in log_path.glob('*.jsonl'):
        data = get_log_data(log_file, earliest)
        if data is not None:
#             print(data)
            log_files.append(data)
    return log_files

In [ ]:
log_files = get_logs(paths.top_level / 'logs-gcp1', earliest = datetime.datetime(2018, 5, 2))

In [ ]:
import pandas as pd
' '.join(pd.DataFrame([data for data in log_files if data.get('complete')]).sort_values('timestamp').participant_id)

In [ ]:
# Dump a list of participant_ids
for config, group in toolz.groupby('config', completed_participants).items():
    print()
    group = sorted(group, key=lambda x: x['timestamp'])
    print(f'{len(group)} completed in {config}')
    print(f'{config}:',  ' '.join(participant['participant_id'] for participant in group))